In [6]:
# extract all slivers
slivers = []
max = 0
with open('../ccle-outputs/sliver-sequences.txt') as file:
    for line in file:
        seq = line.strip()
        if len(seq) > max:
            max = len(seq)
        slivers.append(seq)

print(max)
print(len(slivers))

999
679


In [7]:
# find the kmer value at which repetitive sub-seqs start 
lengths = list(range(max))[::-1]
for i in lengths:
    temp = []
    for seq in slivers:
        if len(seq) >= 10:
            temp.append(seq)

    kmers = []
    for seq in temp:
        for j in range(len(seq) - i + 1):
            kmers.append(seq[j:j+i])

    if len(kmers) != len(set(kmers)):
        print(i)
        break

804


In [8]:
temp_slivers = [slivers[0]]
all_reps = {}
for num, seq in enumerate(slivers):
    rep_seqs = []
    max_len = int(len(seq) / 2)
    lengths = list(range(10, max_len+1))[::-1]
    for i in lengths:
        kmer_sets = int(len(seq) % i) + 1
        for k in range(kmer_sets):
            kmers = {}
            for j in range(k, len(seq)-1, i):
                kmer = seq[j:j+i]
                if kmer in kmers.keys():
                    kmers[seq[j:j+i]] += 1
                else:
                    kmers[seq[j:j+i]] = 1
                kmers_set = list(set(kmers.values()))
            for k, v in kmers.items():
                if v > 1:
                    rep_seqs.append((k,v))
    all_reps[num] = rep_seqs


In [9]:
# save file with all repetitive seqs 
with open('all_reps.tsv', 'w') as file:
    for k,v in all_reps.items():
        if len(v) != 0:
            file.write(str(k) + '\t' + str(v) + '\n')

In [11]:
import pandas as pd
df = pd.read_csv('../ccle-outputs/final-slivers.tsv', sep='\t')
seqs = [k for k in all_reps.keys() if (len(all_reps[k]) != 0)]
sub_df = df.iloc[seqs]
sub_df['rep_seqs'] = [v for v in all_reps.values() if (len(v) != 0)]
sub_df
sub_df.to_csv('rep_seqs.tsv', sep='\t')

/var/folders/kp/8dmz5szd6xsbrx0qpbstvl6h0000gn/T/ipykernel_77370/233684778.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['rep_seqs'] = [v for v in all_reps.values() if (len(v) != 0)]


In [16]:
sub_reps = {}
for k,v in all_reps.items():
    if len(v) != 0:
        sub_reps[k] = v
sub_reps

max_dict = {}
for k,v in sub_reps.items():
    max_tup_num = 0
    max_tup = ()
    for tup in v:
        if tup[1] > max_tup_num:
            max_tup_num = tup[1]
            max_tup = tup
    max_dict[k] = max_tup
max_dict

max_df = sub_df[['Sample', 'Chromosome Position', 'Start Position', 'End Position', 'Sequence']]
max_df['Max Repetitive Seq'] = [v for v in max_dict.values()]
max_df.to_csv('max_rep_seqs.tsv', sep='\t')
max_df['Max Repetitive Seq']

8                                   (GTCGTAAAATACCGA, 3)
10                                      (TACCGACGTAT, 4)
12                          (GGTGTCGTAAAATACCGACGTAT, 2)
13                                      (TGTCGTAAAAT, 2)
14                          (GGTGTCGTAAAATACCGACGTAT, 2)
                             ...                        
639                                    (TGCATTCATCTC, 2)
642                                 (CCTTTGAGCAAATCA, 2)
649                                      (CTACTGTGTG, 2)
663    (CCTCAATCTAGAACATCAGCTCCTCTCCGGGTCTGCAGCTGCATG...
668                                      (CACAGCCCTG, 2)
Name: Max Repetitive Seq, Length: 108, dtype: object